In [1]:
import zipfile
import pandas as pd
import csv
import json
import datetime
import sqlite3

In [54]:
census_df = pd.read_csv('../census-data/FinalBlockGroupData.csv')

In [3]:
weather_zip = zipfile.ZipFile("../weather-data/crime-weather-data.zip", mode='r')
weather_json = weather_zip.open('crime-weather-data.json')
weather_dict = json.load(weather_json)

In [4]:
crime_zip = zipfile.ZipFile("../dc-crime-data/dc-crime-data.csv.zip", mode='r')
crime_csv = crime_zip.open('dc-crime-data.csv')
crime_df = pd.read_csv(crime_csv)

In [53]:
crime_df

,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,WARD,...,BLOCK,START_DATE,CCN,OFFENSE,OCTO_RECORD_ID,ANC,REPORT_DAT,METHOD,location,LATITUDE
0,cluster 5,5500.0,property,-77.052353,2009-03-30T15:50:00.000,theft/other,evening,137490.0,2.0,2.0,...,2400 - 2499 block of m street nw,2009-03-30T05:30:00.000,9041857,theft/other,09041857-01,2A,2009-03-30T20:40:00.000Z,others,"38.905263228683658,-77.052355683742746",38.905255
1,cluster 4,202.0,property,-77.062857,2009-03-30T16:30:00.000,theft/other,evening,137621.0,2.0,2.0,...,1224 - 1299 block of wisconsin avenue nw,2009-03-30T16:20:00.000,9041858,theft/other,09041858-01,2E,2009-03-30T20:34:00.000Z,others,"38.906438143434897,-77.062859682010298",38.906430
2,cluster 17,1804.0,violent,-77.027957,2009-03-30T00:00:00.000,robbery,evening,143785.0,4.0,4.0,...,5900 - 5999 block of georgia avenue nw,2009-03-30T17:00:00.000,9041900,robbery,09041900-01,4A,2009-03-30T21:42:00.000Z,others,"38.961978891670213,-77.027959395242561",38.961971
3,cluster 14,1002.0,property,-77.076820,2009-03-30T07:30:00.000,motor vehicle theft,evening,141241.0,2.0,3.0,...,3810 - 3899 block of rodman street nw,2009-03-29T07:00:00.000,9041916,motor vehicle theft,09041916-01,3C,2009-03-30T22:52:00.000Z,others,"38.939039936788873,-77.076822116815663",38.939032
4,cluster 23,8904.0,property,-76.979941,2009-03-30T20:30:00.000,theft/other,evening,137077.0,5.0,5.0,...,1600 - 1699 block of maryland avenue ne,2009-03-30T18:40:00.000,9041927,theft/other,09041927-01,5D,2009-03-30T22:40:00.000Z,others,"38.901552788657312,-76.979942787227358",38.901545
5,cluster 25,8301.0,property,-76.998986,2009-04-03T00:00:00.000,burglary,midnight,136248.0,1.0,6.0,...,500 - 599 block of c street ne,2009-04-03T00:20:00.000,9044142,burglary,09044142-01,6C,2009-04-03T04:20:00.000Z,others,"38.894086572097208,-76.998987886091982",38.894079
6,cluster 7,4901.0,property,-77.024987,2009-04-04T23:01:00.000,theft/other,midnight,138469.0,3.0,6.0,...,900 - 999 block of s street nw,2009-04-03T14:30:00.000,9044161,theft/other,09044161-01,6E,2009-04-05T03:01:00.000Z,others,"38.914091457382,-77.024988819490929",38.914084
7,cluster 8,5900.0,violent,-77.013660,2009-04-04T00:25:00.000,assault w/dangerous weapon,midnight,136650.0,1.0,6.0,...,500 - 721 block of 2nd street nw,2009-04-03T23:58:00.000,9044204,assault w/dangerous weapon,09044204-01,6C,2009-04-04T06:16:00.000Z,knife,"38.897707140743471,-77.013662755645228",38.897699
8,cluster 8,5800.0,property,-77.022433,2009-04-04T00:30:00.000,theft/other,midnight,136733.0,1.0,2.0,...,700 - 799 block of g street nw,2009-04-04T00:15:00.000,9044207,theft/other,09044207-01,2C,2009-04-04T05:00:00.000Z,others,"38.898453479319308,-77.022435665117129",38.898446
9,cluster 9,11000.0,property,-77.016411,2009-04-03T23:00:00.000,theft/other,midnight,134292.0,1.0,6.0,...,200 - 399 block of m street sw,2009-04-03T22:50:00.000,9044281,theft/other,09044281-01,6D,2009-04-04T07:22:00.000Z,others,"38.876465024278872,-77.016413017836499",38.876457


In [5]:
def convertTime(row):
    time = datetime.datetime.fromtimestamp(row).strftime('%Y-%m-%dT%H:%M:%S.000')

    return time

In [6]:
for w_dict in weather_dict:
    for w_entry in list(w_dict.items()):
        if w_entry[0] == 'currently':
            for items in w_entry[1].items():
                w_dict[items[0]] = items[1]

            del w_dict['currently']

#Create Pandas DataFrame from weather dictionary.
weather_df = pd.DataFrame(weather_dict)

#There are negative time values.
weather_df = weather_df[weather_df['time'] > 0]

#Convert time to format shared by crime data.
weather_df['c_time'] = weather_df['time'].apply(convertTime)

In [ ]:
columns = ['TotalPop','TPopMargin','UnWgtSampleCtPop','PerCapitaIncome','PerCapIncMargin','MedianHouseholdInc',
           'MedHouseholdIncMargin','MedianAge','MedianAgeMargin','HousingUnits','HousingUnitsMargin',
           'UnweightedSampleHousingUnits']

for col in columns:
    mean = census_df[census_df[col] > 0][col].mean()

    census_df.fillna(value={col: mean}, inplace=True)

    census_df[census_df[col] < 0] = mean

census_df['BlockGroup'] = census_df['BlockGroup'].astype(str).replace(']]', '', regex=True)
census_df['Tract'] = census_df['Tract'].astype(str).replace('\.0', '', regex=True)
census_df['Tract'] = census_df['Tract'].apply(lambda x: x.zfill(6))
census_df['BlockGroup'] = census_df['BlockGroup'].astype(str).replace('\.0', '', regex=True)
census_df['Year'] = census_df['Year'].astype(str).replace('\.0', '', regex=True)

census_df['index'] = census_df['Tract'] + " " + census_df['BlockGroup'] + " " + census_df['Year']
census_df_nodup = census_df.drop_duplicates(subset='index')

census_df_nodup.drop(labels=['Unnamed: 0'], axis='columns', inplace=True)

In [86]:
census_df['test'] = census_df['BlockGroup'].astype(str)

In [87]:
census_df['test'][census_df['test'].str.contains('\.0', regex=True)]

Series([], Name: test, dtype: object)

In [57]:
census_df['BlockGroup'].str.__dir__()

['_is_categorical',
 '_parent',
 '_orig',
 '__frozen',
 '__module__',
 '__doc__',
 '__init__',
 '_validate',
 '__getitem__',
 '__iter__',
 '_wrap_result',
 '_get_series_list',
 'cat',
 'split',
 'rsplit',
 'partition',
 'rpartition',
 'get',
 'join',
 'contains',
 'match',
 'replace',
 'repeat',
 'pad',
 'center',
 'ljust',
 'rjust',
 'zfill',
 'slice',
 'slice_replace',
 'decode',
 'encode',
 'strip',
 'lstrip',
 'rstrip',
 'wrap',
 'get_dummies',
 'translate',
 'count',
 'startswith',
 'endswith',
 'findall',
 'extract',
 'extractall',
 'find',
 'rfind',
 'normalize',
 'index',
 'rindex',
 'len',
 'lower',
 'upper',
 'title',
 'capitalize',
 'swapcase',
 'isalnum',
 'isalpha',
 'isdigit',
 'isspace',
 'islower',
 'isupper',
 'istitle',
 'isnumeric',
 'isdecimal',
 '_make_accessor',
 '_freeze',
 '__setattr__',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new

In [19]:
census_df[census_df['TPopMargin'] == False]

,Unnamed: 0,TotalPop,TPopMargin,UnWgtSampleCtPop,PerCapitaIncome,PerCapIncMargin,MedianHouseholdInc,MedHouseholdIncMargin,MedianAge,MedianAgeMargin,HousingUnits,HousingUnitsMargin,UnweightedSampleHousingUnits,State,County,Tract,BlockGroup,Year


In [22]:
census_df[census_df['UnWgtSampleCtPop'] == np.nan]

,Unnamed: 0,TotalPop,TPopMargin,UnWgtSampleCtPop,PerCapitaIncome,PerCapIncMargin,MedianHouseholdInc,MedHouseholdIncMargin,MedianAge,MedianAgeMargin,HousingUnits,HousingUnitsMargin,UnweightedSampleHousingUnits,State,County,Tract,BlockGroup,Year
